In [2]:
import pandas as pd
import networkx as nx
from tqdm import tqdm
from datetime import datetime, timedelta
import classes.transportnetwork as tn
import networkviz.visualisation as viz
import preprocessing.Preprocessing as pp

In [15]:
df_stop_time = pd.read_csv('../../../data/gtfs/stop_times.txt')
df_stop = pd.read_csv('../../../data/gtfs/stops.txt')
df_trip = pd.read_csv('../../../data/gtfs/trips.txt')
df_shape = pd.read_csv('../../../data/gtfs/shapes.txt')

In [16]:
df_stop_time

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint
0,L3334,05:59:00,05:59:00,184:0,1,NaN,0,1,NaN,1
1,L3334,06:04:00,06:04:00,189,2,NaN,0,0,NaN,1
2,L3334,06:07:00,06:07:00,181,3,NaN,0,0,NaN,1
3,L3334,06:08:00,06:08:00,9300,4,NaN,0,0,NaN,1
4,L3334,06:10:00,06:10:00,305,5,NaN,0,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...
37146,N786,19:30:00,19:30:00,580,10,NaN,0,0,NaN,1
37147,N786,19:32:00,19:32:00,1494,11,NaN,0,0,NaN,1
37148,N786,19:35:00,19:35:00,1500,12,NaN,0,0,NaN,1
37149,N786,19:36:00,19:36:00,1499,13,NaN,0,0,NaN,1


In [17]:
df_stop

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,wheelchair_boarding,platform_code,city_name,ext_siri2_id
0,145,HECHA2,Abadia,Direction ~ Norabidea: Plaza San Juan / Grande...,43.374375,-1.752050,NaN,NaN,0,NaN,0,NaN,NaN,NaN
1,64,HECHA1,Abadia,Direction ~ Norabidea: EHPAD / Autoport / Halt...,43.374653,-1.751877,NaN,NaN,0,NaN,0,NaN,NaN,NaN
2,137,AERO01,Aéroport de Biarritz,Direction ~ Norabidea: Bayonne Place des Basqu...,43.472178,-1.532808,NaN,NaN,0,NaN,0,NaN,NaN,137
3,433,HEAGE1,Agerrea,Direction ~ Norabidea: Autoport,43.372023,-1.746788,NaN,NaN,0,NaN,0,NaN,NaN,NaN
4,434,HEAGE2,Agerrea,Direction ~ Norabidea: Grande Plage,43.372303,-1.747608,NaN,NaN,0,NaN,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,1413,HEZO12,Zopite 1,Direction ~ Norabidea: Grande Plage,43.370157,-1.755397,NaN,NaN,0,NaN,0,NaN,NaN,NaN
461,1412,HEZO22,Zopite 2,Direction ~ Norabidea: Grande Plage,43.367703,-1.750537,NaN,NaN,0,NaN,0,NaN,NaN,NaN
462,585,HEZO21,Zopite 2,Direction ~ Norabidea: Autoport,43.367703,-1.750537,NaN,NaN,0,NaN,0,NaN,NaN,NaN
463,1454,CIZUB2,Zubiburu,Direction ~ Norabidea: Lafiténia / Alturan,43.382490,-1.666277,NaN,NaN,0,NaN,0,NaN,NaN,NaN


In [18]:
df_trip

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,67,L327,L3334,Bayonne Place des Basques,0,NaN,L303100,0,0
1,67,L340,L3466,Bayonne Place des Basques,0,NaN,L303100,0,0
2,67,L329,L3335,Bayonne Place des Basques,0,NaN,L303100,0,0
3,67,L338,L3467,Bayonne Place des Basques,0,NaN,L303100,0,0
4,67,L327,L3336,Bayonne Place des Basques,0,NaN,L303101,0,0
...,...,...,...,...,...,...,...,...,...
1335,158,N58,N997,Halte Routière,1,NaN,NaN,0,0
1336,158,N58,N1032,Ahetze,1,NaN,NaN,0,0
1337,158,N3,N270,Ahetze,1,NaN,NaN,0,0
1338,158,N32,N539,Ahetze,1,NaN,NaN,0,0


In [19]:
df_shape

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,L303100,43.372982,-1.775514,1,0.0
1,L303100,43.372959,-1.775670,2,12.0
2,L303100,43.372139,-1.775770,3,104.0
3,L303100,43.372261,-1.774700,4,192.0
4,L303100,43.371929,-1.774640,5,229.0
...,...,...,...,...,...
17463,L303207,43.386742,-1.658680,1606,28251.0
17464,L303207,43.386669,-1.658810,1607,28264.0
17465,L303207,43.386501,-1.659210,1608,28302.0
17466,L303207,43.386269,-1.659940,1609,28366.0


In [20]:
# Merge the two dataframes by keeping the stop sequence order
global_transport_df = pd.merge(df_stop_time, df_stop, on='stop_id', how='left', sort=False)
global_transport_df = pd.merge(global_transport_df, df_trip, on='trip_id', how='left', sort=False)
global_transport_df

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,...,city_name,ext_siri2_id,route_id,service_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,L3334,05:59:00,05:59:00,184:0,1,NaN,0,1,NaN,1,...,NaN,184:0,67,L327,Bayonne Place des Basques,0,NaN,L303100,0,0
1,L3334,06:04:00,06:04:00,189,2,NaN,0,0,NaN,1,...,NaN,189,67,L327,Bayonne Place des Basques,0,NaN,L303100,0,0
2,L3334,06:07:00,06:07:00,181,3,NaN,0,0,NaN,1,...,Hendaye,181,67,L327,Bayonne Place des Basques,0,NaN,L303100,0,0
3,L3334,06:08:00,06:08:00,9300,4,NaN,0,0,NaN,1,...,NaN,9300,67,L327,Bayonne Place des Basques,0,NaN,L303100,0,0
4,L3334,06:10:00,06:10:00,305,5,NaN,0,0,NaN,1,...,NaN,305,67,L327,Bayonne Place des Basques,0,NaN,L303100,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37146,N786,19:30:00,19:30:00,580,10,NaN,0,0,NaN,1,...,NaN,NaN,158,N45,Ahetze,1,NaN,NaN,0,0
37147,N786,19:32:00,19:32:00,1494,11,NaN,0,0,NaN,1,...,NaN,NaN,158,N45,Ahetze,1,NaN,NaN,0,0
37148,N786,19:35:00,19:35:00,1500,12,NaN,0,0,NaN,1,...,NaN,NaN,158,N45,Ahetze,1,NaN,NaN,0,0
37149,N786,19:36:00,19:36:00,1499,13,NaN,0,0,NaN,1,...,NaN,NaN,158,N45,Ahetze,1,NaN,NaN,0,0


In [21]:
G = pp.create_network_from_GTFS('../../../data/gtfs')

Network creation: 


100%|██████████| 37151/37151 [00:08<00:00, 4263.48it/s]


In [ ]:
TN = tn.TransportNetwork(G, pos_argument=['lon', 'lat'])
print(TN)

In [ ]:
viz.map_network(TN)

In [28]:
df_shape = pd.read_csv('../../../data/gtfs_2/shapes.txt')
df_shape

,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,L303100,43.372982,-1.775514,1,0.0
1,L303100,43.372959,-1.775670,2,12.0
2,L303100,43.372139,-1.775770,3,104.0
3,L303100,43.372261,-1.774700,4,192.0
4,L303100,43.371929,-1.774640,5,229.0
...,...,...,...,...,...
65668,N51_202,43.386742,-1.658680,1155,21102.0
65669,N51_202,43.386669,-1.658810,1156,21116.0
65670,N51_202,43.386501,-1.659210,1157,21153.0
65671,N51_202,43.386269,-1.659940,1158,21217.0


In [25]:
df_temp = df_trip.groupby('route_id').first()
df_temp

,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
route_id,,,,,,,,
67,L327,L3334,Bayonne Place des Basques,0,NaN,L303100,0,0
147,N3,N11,Autoport,0,NaN,None,0,0
148,N3,N2,Halte Routière,0,NaN,None,0,0
150,N32,N290,Vieux Fort,0,NaN,None,0,0
151,N32,N289,Zaldi Xurito,0,NaN,None,0,0
152,N3,N1,Urrugne - Bourg,0,NaN,None,0,0
153,N9,N16,EHPAD,0,NaN,None,0,0
154,N3,N13,Béhobia,0,NaN,None,0,0
155,N33,N287,Halte Routière,0,NaN,None,0,0


In [29]:
df_shape_line = df_shape.groupby('shape_id')['shape_pt_lat', 'shape_pt_lon'].apply(lambda x: x.values.tolist())
df_shape_line

/tmp/ipykernel_83586/3756769074.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_shape_line = df_shape.groupby('shape_id')['shape_pt_lat', 'shape_pt_lon'].apply(lambda x: x.values.tolist())


shape_id
L303100    [[43.37298203, -1.77551413], [43.37295914, -1....
L303101    [[43.38620758, -1.66013181], [43.38616943, -1....
L303105    [[43.38620758, -1.66013181], [43.38616943, -1....
L303106    [[43.37298203, -1.77551413], [43.37295914, -1....
L303200    [[43.49356461, -1.47889817], [43.4930687, -1.4...
L303201    [[43.49356461, -1.47889817], [43.4930687, -1.4...
L303202    [[43.49356461, -1.47889817], [43.4930687, -1.4...
L303203    [[43.49356461, -1.47889817], [43.4930687, -1.4...
L303205    [[43.49356461, -1.47889817], [43.4930687, -1.4...
L303207    [[43.49356461, -1.47889817], [43.4930687, -1.4...
N31_100    [[43.33934021, -1.78903949], [43.33932877, -1....
N31_200    [[43.37437057, -1.75203657], [43.37429047, -1....
N31_201    [[43.37637329, -1.75685751], [43.37633133, -1....
N33_100    [[43.37305069, -1.77493739], [43.37302017, -1....
N33_101    [[43.37305069, -1.77493739], [43.37302017, -1....
N33_200    [[43.34295654, -1.75969696], [43.3428688, -1.7...
N33_201    [[43

In [34]:
import plotly.graph_objects as go

fig = go.Figure()

for shape_id, shape in df_shape_line.items():
    fig.add_trace(go.Scattermapbox(
        mode = "lines",
        lon = shape[:,1],
        lat = shape[:,0],
        line = dict(width = 2, color = 'blue'),
        name = shape_id
    ))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 43.38, 'lat': 1.66},
        'style': "stamen-terrain",
        'zoom': 10})

fig.show()

TypeError: list indices must be integers or slices, not tuple